# Fairlearn Bias Evaluation Demo

This notebook demonstrates **Fairlearn bias evaluation techniques** using the public UCI Adult Income dataset.

- The techniques shown here are applicable to **AI fairness evaluation** in domains such as AV safety, public policy, and national security contexts.
- The UCI Adult Income dataset is used here as a standard public example for demonstrating fairness metrics.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate, demographic_parity_difference, equalized_odds_difference


In [ ]:
# Load UCI Adult Income dataset
adult = fetch_openml(data_id=1590, as_frame=True)
df = adult.frame
df.head()

In [ ]:
# Preprocess data
# Use 'sex' as sensitive feature, 'class' as target
df = df.dropna()
X = df.drop(columns=['class'])
y = (df['class'] == '>50K').astype(int)  # Binary target
sensitive_feature = X['sex']

# Encode categorical features
X = X.apply(lambda col: LabelEncoder().fit_transform(col) if col.dtype == 'object' else col)

# Split data
X_train, X_test, y_train, y_test, sf_train, sf_test = train_test_split(
    X, y, sensitive_feature, test_size=0.3, random_state=42)


In [ ]:
# Train simple Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc:.2f}')

In [ ]:
# Evaluate fairness metrics
metric_frame = MetricFrame(
    metrics={
        'accuracy': accuracy_score,
        'selection_rate': selection_rate
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sf_test
)
print(metric_frame.by_group)

dp_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=sf_test)
eo_diff = equalized_odds_difference(y_test, y_pred, sensitive_features=sf_test)

print(f'\nDemographic Parity Difference: {dp_diff:.2f}')
print(f'Equalized Odds Difference: {eo_diff:.2f}')